In [1]:
import cv2
import mediapipe as mp
import numpy as np
import requests
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

In [3]:
# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
for i in range(1, 2):
    cap = cv2.VideoCapture(r"D:\DATASET\1_3000 dataset\KETI_SL_000000000%d.avi" %i)
    with mp_face_mesh.FaceMesh(min_detection_confidence=0.5,min_tracking_confidence=0.5) as face_mesh:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                cv2.destroyAllWindows()
                break
            # the BGR image to RGB.
            image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            results = face_mesh.process(image)
            # Draw the face mesh annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    mp_drawing.draw_landmarks(
                        image=image,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACE_CONNECTIONS,
                        landmark_drawing_spec=drawing_spec,
                        connection_drawing_spec=drawing_spec)
                    #4개 랜드마크의 픽셀 좌표 지정
                    top = int(results.multi_face_landmarks[0].landmark[10].y*720)
                    bottom = int(results.multi_face_landmarks[0].landmark[152].y*720)
                    left = int(results.multi_face_landmarks[0].landmark[234].x*1280) - 40
                    right = int(results.multi_face_landmarks[0].landmark[454].x*1280) + 40
                    #마스크 이미지의 크기 정하기
                    h_mask = int((bottom - top)/2)+45
                    w_mask = right - left
                    #마스크 이미지 로드
                    mask = cv2.imread(r"D:\DATASET\mask.png")
                    alpha = cv2.imread(r"D:\DATASET\alpha.png")
                    mask = mask[350:890, 0:1200]
                    #마스크 크기 조정
                    mask = cv2.resize(mask, (w_mask, h_mask))
                    alpha = cv2.resize(alpha, (w_mask, h_mask))
                    #ROI 추출
                    ROI = image[bottom-h_mask:bottom, left:right]
                    #RGB -> binary
                    ret, mask = cv2.threshold(alpha, 1, 255, cv2.THRESH_BINARY)
                    ROI = cv2.bitwise_and(ROI, ROI, mask = mask)
                    image[bottom-h_mask:bottom, left:right] = ROI
            cv2.imshow('MediaPipe FaceMesh', image)
            if cv2.waitKey(1) & 0xFF == 27:
                cv2.destroyAllWindows()
                break
        cap.release()

error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\core\src\arithm.cpp:250: error: (-215:Assertion failed) (mtype == CV_8U || mtype == CV_8S) && _mask.sameSize(*psrc1) in function 'cv::binary_op'


In [6]:
mask = cv2.imread(r"D:\DATASET\nmask.png", cv2.IMREAD_UNCHANGED)
*_, alpha = cv2.split(mask)
cv2.imwrite("n_alpha.png", alpha)
cv2.waitKey(0)
cv2.destroyAllWindows()